In [60]:
import os
import sys
import csv
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

repo_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
sys.path.append(repo_root)

from src.utils.file_handlers import read_file_parameters


In [85]:
def convergence_file_handlers():
    data_path = os.path.join(os.getcwd(), 'data')

    all_results = []
    for file in os.listdir(data_path):
        if file.endswith('.csv'):
            result = read_file_parameters(file)
            result['avg_HO_iteration'] = 1000
            result['avg_PL_iteration'] = 1000

            file_path = os.path.join(data_path, file)
            df = pd.read_csv(file_path)
            #first iteration error is 0 
            df = df.iloc[1:]

            row_ho = df[df['Avg_HO_Error'] < 1e-10].head(1)
            if not row_ho.empty:
                result['avg_HO_iteration'] = row_ho['Iteration'].values[0]
            
            # Find the first row where Avg_PL_Error is below the threshold
            row_pl = df[df['Avg_PL_Error'] < 1e-10].head(1)
            if not row_pl.empty:
                result['avg_PL_iteration'] = row_pl['Iteration'].values[0]


            all_results.append(result)


    df = pd.DataFrame(all_results)
    numeric_columns = ['avg_HO_iteration', 'avg_PL_iteration']
    final_df = df.groupby('dataset')[numeric_columns].mean().reset_index()
    final_df['dataset'] = pd.to_numeric(final_df['dataset'], errors='coerce')
    return final_df.sort_values(by='dataset')


def visualize_df(df, title):
    print(f"\n{title}\n")
    styled_df = df.style.set_table_styles(
    ).set_properties(**{
        'background-color': 'LightGray',
        'color': 'black',
        'border-color': 'black',
        'border-style': 'solid',
        'border-width': '1px',
        'text-align': 'left'
    })
    styled_df = styled_df.format(precision=4)

    # print(df.to_latex(escape=True))
    display(styled_df)


In [89]:

df = convergence_file_handlers()
df.to_csv('results/convergence_table.csv')

In [87]:
title = "Convergence Results of Plackett-Luce and generalized BT"
visualize_df(df, title)


Convergence Results of Plackett-Luce and generalized BT



,dataset,avg_HO_iteration,avg_PL_iteration
0,1,1000.0000,1000.0000
8,2,1000.0000,1000.0000
17,4,237.1200,624.8400
23,6,1000.0000,1000.0000
24,7,23.2000,45.3600
25,8,1000.0000,1000.0000
26,9,17.7600,1000.0000
5,12,19.1200,40.1600
6,14,56.0400,86.6800
7,18,1000.0000,1000.0000
